In [1]:
textgrid_dir = "/Users/joregan/Playing/textgrids_shared"
timecode_dir = "/Users/joregan/timecode_cut"

In [2]:
from pathlib import Path

textgrid_path = Path(textgrid_dir)
timecode_path = Path(timecode_dir)

In [3]:
def get_timecode_offsets(filename):
    if type(filename) == Path:
        filename = str(filename)

    with open(filename) as inf:
        lines = [l.strip() for l in inf.readlines()]
        assert lines[0] == ",Frame,Time (Seconds),TimeCode", f"CSV file ({filename}) seems to be incorrect"
        p_start = lines[1].split(",")
        start = float(p_start[2])
        p_end = lines[-1].split(",")
        end = float(p_end[2])
        return start, end

In [5]:
from praatio import textgrid

def modify_textgrid(filename, start, end, output=None):
    if type(filename) == Path:
        filename = str(filename)
    if not output:
        output = filename
    tg = textgrid.openTextgrid(filename, False)
    for tier in tg.tierNames:
        pass

In [6]:
tmp = textgrid.openTextgrid("/Users/joregan/Playing/textgrids_shared/hsi_6_0718_211_001_main.TextGrid", False)

In [10]:
for t in tmp.tierNames:
    print(t)
tier = tmp.getTier("words")
tier.entries

words
phones
utterances


(Interval(start=10.555, end=10.855, label='or'),
 Interval(start=14.4967, end=14.8067, label='yeah'),
 Interval(start=14.8067, end=15.0867, label='yeah'),
 Interval(start=15.0867, end=15.3967, label='yeah'),
 Interval(start=15.7383, end=16.0383, label='ah'),
 Interval(start=16.0383, end=16.0983, label='yeah'),
 Interval(start=16.3479, end=16.4479, label='new'),
 Interval(start=16.4479, end=16.9679, label='apartment'),
 Interval(start=16.9679, end=17.0879, label='new'),
 Interval(start=17.0879, end=17.7479, label='iphone'),
 Interval(start=17.7479, end=17.8079, label='you'),
 Interval(start=17.8079, end=17.9879, label='see'),
 Interval(start=17.9879, end=18.2779, label='that'),
 Interval(start=18.739, end=18.819, label="it's"),
 Interval(start=18.819, end=18.949, label='the'),
 Interval(start=18.949, end=19.679, label='iphone'),
 Interval(start=19.679, end=19.989, label='uh'),
 Interval(start=20.379, end=20.699, label='iphone'),
 Interval(start=20.699, end=20.999, label='twenty'),
 Inte